## Snippets: Usage of filters and metadata 
https://cloud.google.com/retail/docs/filter-and-order
The expressions for "filter" are expected to work in these examples

Metadata for the this store <ES_STORE_ID> looks like this:

"id": "1", "structData": {"title": "Document1", "category": ["PersonaA"], "name": "Document1"}, "content": {"mimeType": "application/pdf", "uri": "gs://<BUCKETNAME>/data/Document1"}}


{"id": "2", "structData": {"title": "Document2", "category": ["PersonaA", "PersonaB"], "name": "Document2"}, "content": {"mimeType": "application/pdf", "uri": "gs://<BUCKETNAME>/data/Document2"}}

    

### Below the filter name: ANY(..) matches ensure the query is against only the documents' metadata satisfying that criteria.

In [ ]:
%%bash
curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search:search" \
-d '{
"servingConfig": "projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search",
"query": "claim",
"filter": "name: ANY(\"Document1\")"
}'

### Below the filter name: ANY(..) matches ensure the query is against only the documents' metadata satisfying that criteria.

In [ ]:
%%bash
curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search:search" \
-d '{
"servingConfig": "projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search",
"query": "claims",
"filter": "category: ANY(\"PersonaB\")"
}'

### Python Code equivalent

In [ ]:
# Local import of retriever class
import sys, os
sys.path.append(os.path.abspath(".."))
from utils.es import es_raw_search_summary

In [ ]:
PROJECT_ID = "<PROJECT>"
SEARCH_ENGINE_ID = "<ES_STORE_ID>"

In [ ]:
query = "how to process a claim"
rawresults = None
summary, rawresults = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query, filtername="name: ANY(\"Document1\")")
#summary, rawresult = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query)
print(f"\nQuestion: '{query}'\n\nES Summary: '{summary}'")
print("Rawresult" + "-"*40)
print(rawresults)
print("Summary" + "-"*40)
print(summary)

## In this example I set a slightly more complex filter based on 2 metadata values

In [ ]:
query = "how to process a claim"
rawresults = None
filtersample = "name: ANY(\"Document1\") AND category: ANY(\"PersonaA\")"
summary, rawresults = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query, filtername=filtersample)
#summary, rawresult = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query)
print(f"\nQuestion: '{query}'\n\nES Summary: '{summary}'")
print("Rawresult" + "-"*40)
print(rawresults)
print("Summary" + "-"*40)
print(summary)